# NFL Fantasy Football Rookie Success Analysis


### This notebook is designed to be run sequentially. While all cells should generally be executed, the "college data retrieval" section contains optional blocks for data validation. Markdown cells will explicitly identify the code required for the final analysis.

## Install necessary packages and grab initial data


In [20]:
# For the data ingestion and cleaning tasks, these 4 imports are all you need. nfl_data_py is available via pip install. 
# Please use a version of python that is 3.12 or earlier, as the nfl_data_py would fail to build wheel dependecies on 3.13+.

import nfl_data_py as nfl
import os
import pandas as pd
import requests

## Task 1: NFL Draft Results (2015 - 2025)

In [21]:
#NFL Draft Results (2025)
picks_2015_2025 = nfl.import_draft_picks(range(2015, 2026))
#picks_2025.columns

filtered_picks_2015_2025 = picks_2015_2025[['season', 'round', 'pick', 'team', 'pfr_player_id', 'pfr_player_name', 'college', 'position']]

#Print the results
filtered_picks_2015_2025

season  round  pick team pfr_player_id   pfr_player_name       college  \
9849     2015      1     1  TAM      WinsJa00    Jameis Winston   Florida St.   
9850     2015      1     2  TEN      MariMa01    Marcus Mariota        Oregon   
9851     2015      1     3  JAX      FowlDa00      Dante Fowler       Florida   
9852     2015      1     4  OAK      CoopAm00      Amari Cooper       Alabama   
9853     2015      1     5  WAS      ScheBr00   Brandon Scherff          Iowa   
...       ...    ...   ...  ...           ...               ...           ...   
12665    2025      7   253  MIA      BiggZe01      Zeek Biggers  Georgia Tech   
12666    2025      7   254  NOR      DiggFa00       Fadil Diggs      Syracuse   
12667    2025      7   255  HOU      LachLu00       Luke Lachey          Iowa   
12668    2025      7   256  LAC      BridTr00  Trikweze Bridges       Florida   
12669    2025      7   257  NWE      MinoKo00       Kobee Minor       Memphis   

      position  
9849        QB  
9850        QB  
9851       OLB  
9852        WR  
9853         T  
...        ...  
12665       DL  
12666       DE  
12667       TE  
12668      SAF  
12669       DB  

[2821 rows x 8 columns]

## Task 2: Combine Results (2015 - 2025)

In [22]:
#Combine Results (2010-2025)
combine_results_2015_2025 = nfl.import_combine_data(range(2015, 2026))

#There are 5062 rows of data
#len(combine_results_2010_2025)

#combine_results_2010_2025

#Columns to Look At: ht, wt, forty, bench, vertical, broad_jump, cone, and shuttle
combine_results_2015_2025.head()

season  draft_year            draft_team  draft_round  draft_ovr  \
4902    2015      2015.0         Detroit Lions          2.0       54.0   
4903    2015      2015.0   Philadelphia Eagles          1.0       20.0   
4904    2015      2015.0        Miami Dolphins          5.0      149.0   
4905    2015      2015.0  Tampa Bay Buccaneers          4.0      124.0   
4906    2015      2015.0    Cincinnati Bengals          7.0      238.0   

        pfr_id            cfb_id     player_name  pos         school   ht  \
4902  AbduAm00  ameer-abdullah-1  Ameer Abdullah   RB       Nebraska  5-9   
4903  AghoNe00  nelson-agholor-1  Nelson Agholor   WR            USC  6-0   
4904  AjayJa00       jay-ajayi-1       Jay Ajayi   RB    Boise State  6-0   
4905  AlexKw00  kwon-alexander-1  Kwon Alexander  OLB            LSU  6-1   
4906  AlfoMa00    mario-alford-1    Mario Alford   WR  West Virginia  5-8   

         wt  forty  bench  vertical  broad_jump  cone  shuttle  
4902  205.0   4.60   24.0      42.5       130.0  6.79     3.95  
4903  198.0   4.42   12.0       NaN         NaN   NaN      NaN  
4904  221.0   4.57   19.0      39.0       121.0  7.10     4.10  
4905  227.0   4.55   24.0      36.0       121.0  7.14     4.20  
4906  180.0   4.43   13.0      34.0       121.0  6.64     4.07

## Task 3: College Football Stats (2010-2025)

In [23]:
#API Key: vAxpOlW6/4/6GwI44zGXcenYCXf5AFP6fJ5wxYwkotfP9nQ3QYJwlG6rl+wOUiHR

In [24]:
API_KEY = "vAxpOlW6/4/6GwI44zGXcenYCXf5AFP6fJ5wxYwkotfP9nQ3QYJwlG6rl+wOUiHR"
headers = {"Authorization": f"Bearer {API_KEY}"}

In [26]:
url = "https://api.collegefootballdata.com/stats/player/season?year=2024"
resp = requests.get(url, headers=headers)

In [27]:
CFBD_data_2024 = resp.json()
df = pd.json_normalize(CFBD_data_2024)

df.head()

season playerId       player position        team conference   category  \
0    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
1    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
2    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
3    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
4    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   

  statType stat  
0       PD    0  
1   QB HUR    0  
2    SACKS    0  
3     SOLO    1  
4       TD    0

In [28]:
#Unique values of the statType column
df["statType"].unique()

array(['PD', 'QB HUR', 'SACKS', 'SOLO', 'TD', 'TFL', 'TOT', 'FUM', 'LOST',
       'REC', 'ATT', 'COMPLETIONS', 'INT', 'PCT', 'YDS', 'YPA', 'CAR',
       'LONG', 'YPC', 'AVG', 'NO', 'YPR', 'FGA', 'FGM', 'PTS', 'XPA',
       'XPM', 'In 20', 'TB', 'YPP'], dtype=object)

In [29]:
pivot_df = df.pivot_table(
    index=["season", "playerId", "player", "position", "team", "conference", "category"],
    columns="statType",
    values="stat",
    aggfunc="sum"
).reset_index()

pivot_df.head()

statType  season playerId        player position        team conference  \
0           2024   102597   Will Rogers       QB  Washington    Big Ten   
1           2024   102597   Will Rogers       QB  Washington    Big Ten   
2           2024   102597   Will Rogers       QB  Washington    Big Ten   
3           2024   102597   Will Rogers       QB  Washington    Big Ten   
4           2024   107494  Trey Sanders       RB         TCU     Big 12   

statType   category  ATT  AVG  CAR  ...   TD  TFL  TOT  XPA  XPM   YDS  YPA  \
0         defensive  NaN  NaN  NaN  ...    0    0    1  NaN  NaN   NaN  NaN   
1           fumbles  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   NaN  NaN   
2           passing  311  NaN  NaN  ...   14  NaN  NaN  NaN  NaN  2458  7.9   
3           rushing  NaN  NaN   34  ...    0  NaN  NaN  NaN  NaN   -79  NaN   
4         defensive  NaN  NaN  NaN  ...    0    0    1  NaN  NaN   NaN  NaN   

statType   YPC  YPP  YPR  
0          NaN  NaN  NaN  
1          NaN  NaN  NaN  
2          NaN  NaN  NaN  
3         -2.3  NaN  NaN  
4          NaN  NaN  NaN  

[5 rows x 37 columns]

In [30]:
#Quick Sanity Check: Ashton Jeanty Stats in 2024
Ashton_Jeanty_stats_2024 = pivot_df[pivot_df["player"] == "Ashton Jeanty"]
Ashton_Jeanty_stats_2024

statType  season playerId         player position         team     conference  \
13112       2024  4890973  Ashton Jeanty       RB  Boise State  Mountain West   
13113       2024  4890973  Ashton Jeanty       RB  Boise State  Mountain West   
13114       2024  4890973  Ashton Jeanty       RB  Boise State  Mountain West   
13115       2024  4890973  Ashton Jeanty       RB  Boise State  Mountain West   

statType     category  ATT   AVG  CAR  ...   TD  TFL  TOT  XPA  XPM   YDS  \
13112         fumbles  NaN   NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN   NaN   
13113     kickReturns  NaN  11.0  NaN  ...    0  NaN  NaN  NaN  NaN    11   
13114       receiving  NaN   NaN  NaN  ...    1  NaN  NaN  NaN  NaN   138   
13115         rushing  NaN   NaN  374  ...   29  NaN  NaN  NaN  NaN  2601   

statType  YPA  YPC  YPP  YPR  
13112     NaN  NaN  NaN  NaN  
13113     NaN  NaN  NaN  NaN  
13114     NaN  NaN  NaN  6.0  
13115     NaN  7.0  NaN  NaN  

[4 rows x 37 columns]

### The block below is the core block you need to run to create the full list of college data which is used later in the manipualtion and cleaning phase

In [31]:
# Given the amount of data, this code block takes about 3 mins to complete

#Empty list
data_2010_2025 = []

# loop through 2010–2025
# we're only looking at rookies from 2015 onward, however we'll grab college data dating back to 2010 to account for the 4-5 years 2015 rookies would have been in college
for year in range(2010, 2026):
    url = f"https://api.collegefootballdata.com/stats/player/season?year={year}"
    resp = requests.get(url, headers=headers)

    data = resp.json()
    df_year = pd.json_normalize(data)

    data_2010_2025.append(df_year)

# this is the dataframe used in the cleaning and manipulation steps later
df_all = pd.concat(data_2010_2025, ignore_index=True)

df_all.head()

season playerId          player position           team      conference  \
0    2010   500611     Jeff Thomas       LB        Indiana         Big Ten   
1    2010   243266   Joe Bauserman       QB     Ohio State         Big Ten   
2    2010   241724    Shane Popham        P    Wake Forest             ACC   
3    2010   382083  Cameron Graham       TE     Louisville        Big East   
4    2010   184622   Dwayne Harris       WR  East Carolina  Conference USA   

        category statType  stat  
0  interceptions      YDS     0  
1        passing      YDS   174  
2        punting      YDS  2633  
3      receiving       TD     5  
4        rushing       TD     0

In [ ]:
#There are 1292723 rows of data
#len(df_all)

## Task 4: NFL Player Rookie Season Stats (NFL PY) [2010-2025]

In [32]:
# import the players data frame that we'll combine with the season stats so we have the pfr_id to merge with the draft data
players_df = nfl.import_players()
players_df = players_df[['gsis_id', 'display_name', 'pfr_id', 'position', 'college_name', 'rookie_season']]
players_df = players_df[players_df['position'] == 'RB']
players_df = players_df[players_df['rookie_season'] >= 2015]
players_df.head()

gsis_id       display_name    pfr_id position college_name  \
1    00-0038389  Israel Abanikanda  AbanIs00       RB   Pittsburgh   
7    00-0032104     Ameer Abdullah  AbduAm00       RB     Nebraska   
33   00-0039040      De'Von Achane  AchaDe00       RB    Texas A&M   
75   00-0034457         Josh Adams  AdamJo03       RB   Notre Dame   
159  00-0036020       Salvon Ahmed  AhmeSa01       RB   Washington   

     rookie_season  
1             2023  
7             2015  
33            2023  
75            2018  
159           2020

In [33]:
# grab all data from 2015-2024 seasons
season_stats_2015_2024_df = nfl.import_seasonal_data(list(range(2015, 2025)), 'REG')

# select only relevant columns
relevant_cols = ['player_id','season','carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'receptions', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost']

# merge on unique id and rookie year to only grab first year stats
full_season_2015_2024_filtered = players_df.merge(
    season_stats_2015_2024_df[relevant_cols],
    left_on=['gsis_id', 'rookie_season'], right_on=['player_id','season'],
    how='left'
)

full_season_2015_2024_filtered.head()

gsis_id       display_name    pfr_id position college_name  \
0  00-0038389  Israel Abanikanda  AbanIs00       RB   Pittsburgh   
1  00-0032104     Ameer Abdullah  AbduAm00       RB     Nebraska   
2  00-0039040      De'Von Achane  AchaDe00       RB    Texas A&M   
3  00-0034457         Josh Adams  AdamJo03       RB   Notre Dame   
4  00-0036020       Salvon Ahmed  AhmeSa01       RB   Washington   

   rookie_season   player_id  season  carries  rushing_yards  rushing_tds  \
0           2023  00-0038389  2023.0     22.0           70.0          0.0   
1           2015  00-0032104  2015.0    143.0          597.0          2.0   
2           2023  00-0039040  2023.0    103.0          800.0          8.0   
3           2018  00-0034457  2018.0    120.0          511.0          3.0   
4           2020  00-0036020  2020.0     75.0          319.0          3.0   

   rushing_fumbles  rushing_fumbles_lost  receptions  receiving_yards  \
0              0.0                   0.0         7.0             43.0   
1              4.0                   2.0        25.0            183.0   
2              1.0                   1.0        27.0            197.0   
3              1.0                   1.0         7.0             58.0   
4              0.0                   0.0        11.0             61.0   

   receiving_tds  receiving_fumbles  receiving_fumbles_lost  
0            0.0                0.0                     0.0  
1            1.0                0.0                     0.0  
2            3.0                0.0                     0.0  
3            0.0                0.0                     0.0  
4            0.0                0.0                     0.0

## Task 5: Filter Draft Data and Apply Position Rankings

We'll break our analysis down by position - To start, we'll create a general running back dataframe that will take the list of drafted running backs from 2010 to 2025 and bring in their combine results and college stats.

For players drafted before 2025, we will bring in their first NFL season stats to see how they fared in their rookie year.

In [34]:
# Filter draft results for only Running Backs
rb_2015_2025 = filtered_picks_2015_2025[filtered_picks_2015_2025['position'] == 'RB']

# Calculate pick number for the position in each draft
rb_2015_2025 = rb_2015_2025.sort_values(by=['season','pick'])
rb_2015_2025['rank'] = rb_2015_2025.groupby('season').cumcount() + 1
rb_2015_2025['position_rank'] = rb_2015_2025['position'] + rb_2015_2025['rank'].astype(str)
rb_2015_2025.drop(labels='rank', axis='columns', inplace=True)

# there is a sinugular player who's pfr id is null - for simplicity, we will drop them for now
rb_2015_2025 = rb_2015_2025[rb_2015_2025['pfr_player_id'].notna()]

rb_2015_2025.head()

season  round  pick team pfr_player_id pfr_player_name    college  \
9858    2015      1    10  STL      GurlTo01     Todd Gurley    Georgia   
9863    2015      1    15  SDG      GordMe00   Melvin Gordon  Wisconsin   
9884    2015      2    36  JAX      YeldT.00     T.J. Yeldon    Alabama   
9902    2015      2    54  DET      AbduAm00  Ameer Abdullah   Nebraska   
9921    2015      3    73  ATL      ColeTe01   Tevin Coleman    Indiana   

     position position_rank  
9858       RB           RB1  
9863       RB           RB2  
9884       RB           RB3  
9902       RB           RB4  
9921       RB           RB5

## Task 6: Merge Draft and Combine Data

In [35]:
# Add in combine data for each player
# NFL Data Py provides a pfr_id which should make this initial join straightforward

# filter combine dataframe for only the new columns we want to add
combine_columns = ['pfr_id', 'ht', 'wt', 'forty', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle']

# merge dataframes on the pfr_id which has slightly different names between the two frames
rb_2015_2025 = rb_2015_2025.merge(
    combine_results_2015_2025[combine_columns],
    left_on='pfr_player_id', right_on ='pfr_id',
    how='left'
)

# drop the duplicate pfr column from combine dataframe
rb_2015_2025.drop(labels='pfr_id', axis='columns', inplace=True)

# create unique_id column that will be used to join with college career stats
rb_2015_2025['unique_id'] = (
    rb_2015_2025['pfr_player_name'].str.lower().replace(r'[^a-z0-9]', '', regex=True) +
    rb_2015_2025['position'].str.lower() +
    rb_2015_2025['college'].str.lower().replace(r'[^a-z0-9]', '', regex=True)
)

rb_2015_2025.head()

season  round  pick team pfr_player_id pfr_player_name    college position  \
0    2015      1    10  STL      GurlTo01     Todd Gurley    Georgia       RB   
1    2015      1    15  SDG      GordMe00   Melvin Gordon  Wisconsin       RB   
2    2015      2    36  JAX      YeldT.00     T.J. Yeldon    Alabama       RB   
3    2015      2    54  DET      AbduAm00  Ameer Abdullah   Nebraska       RB   
4    2015      3    73  ATL      ColeTe01   Tevin Coleman    Indiana       RB   

  position_rank    ht     wt  forty  bench  vertical  broad_jump  cone  \
0           RB1   6-1  222.0   4.52   17.0       NaN         NaN   NaN   
1           RB2   6-1  215.0   4.52   19.0      35.0       126.0  7.04   
2           RB3   6-1  226.0   4.61   22.0      36.0       117.0  7.19   
3           RB4   5-9  205.0   4.60   24.0      42.5       130.0  6.79   
4           RB5  5-11  206.0   4.40   22.0       NaN         NaN   NaN   

   shuttle                unique_id  
0      NaN      toddgurleyrbgeorgia  
1     4.07  melvingordonrbwisconsin  
2     4.22        tjyeldonrbalabama  
3     3.95  ameerabdullahrbnebraska  
4      NaN    tevincolemanrbindiana

## Task 7: Filter and Aggregate College Data

In [36]:
# Now, we will aggregate career college stats for each player then join with draft and combine data
# We will perform this for just the rushing since that is the most relevant for running backs

# filter for just RBs and rushing stats to make future analysis more efficient
college_rush_df = df_all[df_all['category'].isin(['rushing'])]

# there is a weird nuance in the data where position is sometimes listed as "?" for some players
# since we are looking at rushing both here and in the draft + combine stats, we'll assume those ? are running backs to ensure more matches
college_rush_df['position'] = college_rush_df['position'].str.replace('?', 'RB', regex=False)

college_rush_df = college_rush_df.pivot_table(
    index=["season", "playerId", "player", "position", "team", "conference", "category"],
    columns="statType",
    values="stat",
    aggfunc="sum"
).reset_index()

# drop YPC column for now - we will recalculate after aggregating to career level by dividing yards by number of carries
college_rush_df.drop(labels='YPC', axis='columns', inplace=True)

# rename stat columns to clarify these are college stats
column_mapping = {
    'CAR':'college_carries',
    'LONG':'college_long',
    'TD':'college_tds',
    'YDS':'college_yards'
}

college_rush_df.rename(columns=column_mapping,inplace=True)

# also change naming within column of "state" to "st." to ensure more matches with draft and combine data later on
college_rush_df['team'] = college_rush_df['team'].str.replace('State', 'St.', regex=False)

# ensure that these columns are cast as numeric so aggregation functions succeed
cols_to_convert = ['college_carries', 'college_tds', 'college_yards', 'college_long']
college_rush_df[cols_to_convert] = college_rush_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

/var/folders/gs/h66bgzdj3zs4_ncr3x01_0yw0000gn/T/ipykernel_50211/1220393902.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  college_rush_df['position'] = college_rush_df['position'].str.replace('?', 'RB', regex=False)


In [37]:
# aggregate stats to player level for college career stats
# one nuance we need to account for is the transfer portal - schools in the draft are always the last college they attended

# per player id, we'll sort to the latest season they played
college_data_sorted = college_rush_df.sort_values(by=['playerId', 'season'])

# group by playerId and aggregate
college_career_stats = college_data_sorted.groupby(['playerId', 'player', 'position']).agg(
    # grab the last team name and year - our sort earlier ensures this grabs the last team they played for
    college=('team', 'last'),
    year=('season','max'),

    # stat totals, specific to each category
    college_carries=('college_carries', 'sum'),
    college_tds=('college_tds', 'sum'),
    college_yards=('college_yards', 'sum'),
    college_long=('college_long', 'max')
).reset_index()

# create "unique id" column that can be used to join to draft + combine table
college_career_stats['unique_id'] = (
    college_career_stats['player'].str.lower().replace(r'[^a-z0-9]', '', regex=True) +
    college_career_stats['position'].str.lower() +
    college_career_stats['college'].str.lower().replace(r'[^a-z0-9]', '', regex=True)
)

# so as it turns out, we had a couple instances were a player suddenly had different IDs in the dataset (Cj and CJ names that resulted in different IDs)
# because of this, we'll aggregate one more time based on the unique_id since based on the duplicates, they are intended to be the same player

college_career_stats = college_career_stats.groupby('unique_id').agg(
    college_carries=('college_carries', 'sum'),
    college_tds=('college_tds', 'sum'),
    college_yards=('college_yards', 'sum'),
    college_long=('college_long', 'max')
).reset_index()

#recalculate college ypc now that we are at a career level
college_career_stats['college_ypc'] = round(college_career_stats['college_yards']/college_career_stats['college_carries'],1)

college_career_stats.head()

unique_id  college_carries  college_tds  college_yards  \
0  aanjayfelicianowrstetson                1            0             -4   
1      aarenvaughnswrutahst               18            1            199   
2      aaronallenqbalcornst              139            3             89   
3        aaronandersonwrlsu                1            0              2   
4  aaronangelosqbdelawarest               25            0            -51   

   college_long  college_ypc  
0             0         -4.0  
1            71         11.1  
2            22          0.6  
3             2          2.0  
4             8         -2.0

## Task 8: Merge College & NFL Rookie Season Data

In [38]:
# and now, using the unique id, we'll bring in the career college stats for each player
rb_2015_2025['unique_id'] = rb_2015_2025['unique_id'].str.strip().str.lower()
college_career_stats['unique_id'] = college_career_stats['unique_id'].str.strip().str.lower()

stat_columns = ['college_carries', 'college_tds', 'college_yards', 'college_long', 'college_ypc']

rb_2015_2025 = rb_2015_2025.merge(
    college_career_stats[stat_columns + ['unique_id']],
    on='unique_id',
    how='left'
)

rb_2015_2025.head()

season  round  pick team pfr_player_id pfr_player_name    college position  \
0    2015      1    10  STL      GurlTo01     Todd Gurley    Georgia       RB   
1    2015      1    15  SDG      GordMe00   Melvin Gordon  Wisconsin       RB   
2    2015      2    36  JAX      YeldT.00     T.J. Yeldon    Alabama       RB   
3    2015      2    54  DET      AbduAm00  Ameer Abdullah   Nebraska       RB   
4    2015      3    73  ATL      ColeTe01   Tevin Coleman    Indiana       RB   

  position_rank    ht  ...  vertical  broad_jump  cone  shuttle  \
0           RB1   6-1  ...       NaN         NaN   NaN      NaN   
1           RB2   6-1  ...      35.0       126.0  7.04     4.07   
2           RB3   6-1  ...      36.0       117.0  7.19     4.22   
3           RB4   5-9  ...      42.5       130.0  6.79     3.95   
4           RB5  5-11  ...       NaN         NaN   NaN      NaN   

                 unique_id  college_carries  college_tds college_yards  \
0      toddgurleyrbgeorgia              NaN          NaN           NaN   
1  melvingordonrbwisconsin            631.0         45.0        4915.0   
2        tjyeldonrbalabama            576.0         37.0        3322.0   
3  ameerabdullahrbnebraska            813.0         39.0        4588.0   
4    tevincolemanrbindiana            452.0         28.0        3219.0   

   college_long  college_ypc  
0           NaN          NaN  
1          88.0          7.8  
2          68.0          5.8  
3          62.0          5.6  
4          90.0          7.1  

[5 rows x 23 columns]

In [39]:
stat_cols = ['pfr_id','carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'receptions', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost']

full_rb_2015_2025_df = rb_2015_2025.merge(
    full_season_2015_2024_filtered[stat_cols],
    left_on='pfr_player_id', right_on='pfr_id',
    how='left'
)

# for any nan values in the stat columns, we'll replace with 0 so the fantasy calculation works correctly
full_rb_2015_2025_df[stat_cols] = full_rb_2015_2025_df[stat_cols].fillna(0)


## Task 9: Calculate Full Season Points per Reception (PPR) Fantasy Score

In [40]:
# now, we'll define a function to calculate fantasy points based on ppr scoring system
def calculate_fantasy_points(row):
    points = (
        row['rushing_yards'] * 0.1 +
        row['rushing_tds'] * 6 +
        row['rushing_fumbles_lost'] * -2 +
        row['receiving_yards'] * 0.1 +
        row['receptions'] * 1 +
        row['receiving_tds'] * 6 +
        row['receiving_fumbles_lost'] * -2
    )

    return round(points, 2)

In [41]:
# now we can apply the function and caluclate fantasy points scored for each player in their rookie year
full_rb_2015_2025_df['fantasy_points'] = full_rb_2015_2025_df.apply(calculate_fantasy_points, axis=1)

# let's see who the highest scoring rookie was over these years!
full_rb_2015_2025_df.sort_values(by='fantasy_points', ascending=False).head(20)

season  round  pick team pfr_player_id      pfr_player_name  \
65     2018      1     2  NYG      BarkSa00       Saquon Barkley   
18     2016      1     4  DAL      ElliEz00      Ezekiel Elliott   
42     2017      3    67  NOR      KamaAl00         Alvin Kamara   
129    2021      1    24  PIT      HarrNa00         Najee Harris   
43     2017      3    86  KAN      HuntKa00          Kareem Hunt   
113    2020      2    41  IND      TaylJo02      Jonathan Taylor   
171    2023      1     8  ATL      RobiBi01       Bijan Robinson   
194    2024      4   125  TAM      IrviBu00         Bucky Irving   
172    2023      1    12  DET      GibbJa01         Jahmyr Gibbs   
38     2017      1     4  JAX      FourLe00    Leonard Fournette   
27     2016      5   150  CHI      HowaJo00        Jordan Howard   
39     2017      1     8  CAR      McCaCh01  Christian McCaffrey   
87     2019      2    53  PHI      SandMi01        Miles Sanders   
6      2015      3    86  ARI      JohnDa08        David Johnson   
0      2015      1    10  STL      GurlTo01          Todd Gurley   
131    2021      2    35  DEN      WillJa10     Javonte Williams   
149    2022      2    41  SEA      WalkKe00   Kenneth Walker III   
117    2020      3    66  WAS      GibsAn00       Antonio Gibson   
68     2018      2    35  CLE      ChubNi00           Nick Chubb   
86     2019      1    24  OAK      JacoJo01          Josh Jacobs   

            college position position_rank    ht  ...  rushing_yards  \
65         Penn St.       RB           RB1   6-0  ...         1307.0   
18         Ohio St.       RB           RB1   6-0  ...         1631.0   
42        Tennessee       RB           RB5  5-10  ...          728.0   
129         Alabama       RB           RB1   NaN  ...         1200.0   
43           Toledo       RB           RB6  5-10  ...         1327.0   
113       Wisconsin       RB           RB3  5-10  ...         1169.0   
171           Texas       RB           RB1  5-11  ...          976.0   
194          Oregon       RB           RB6   5-9  ...         1122.0   
172         Alabama       RB           RB2   5-9  ...          945.0   
38              LSU       RB           RB1   6-0  ...         1040.0   
27          Indiana       RB          RB10   6-0  ...         1313.0   
39         Stanford       RB           RB2  5-11  ...          435.0   
87         Penn St.       RB           RB2  5-11  ...          818.0   
6     Northern Iowa       RB           RB7   6-1  ...          581.0   
0           Georgia       RB           RB1   6-1  ...         1106.0   
131  North Carolina       RB           RB3   5-9  ...          903.0   
149    Michigan St.       RB           RB2   5-9  ...         1050.0   
117         Memphis       RB           RB7   6-0  ...          795.0   
68          Georgia       RB           RB4  5-11  ...          996.0   
86          Alabama       RB           RB1  5-10  ...         1150.0   

     rushing_tds  rushing_fumbles  rushing_fumbles_lost  receptions  \
65          11.0              0.0                   0.0        91.0   
18          15.0              5.0                   1.0        32.0   
42           8.0              1.0                   1.0        81.0   
129          7.0              0.0                   0.0        74.0   
43           8.0              1.0                   1.0        53.0   
113         11.0              1.0                   1.0        36.0   
171          4.0              4.0                   3.0        58.0   
194          8.0              2.0                   1.0        47.0   
172         10.0              2.0                   1.0        52.0   
38           9.0              2.0                   0.0        36.0   
27           6.0              1.0                   1.0        29.0   
39           2.0              1.0                   1.0        80.0   
87           3.0              2.0                   1.0        50.0   
6            8.0              2.0                   0.0        36.0  

In [42]:
# let's take this output and export to csv so we can use it in our analysis and visualization steps without having to do the whole loading and manipulation steps again
from pathlib import Path

downloads_path = Path.home() / "Downloads"
file_name = "running_backs_2015_2025.csv"
full_file_path = downloads_path / file_name

full_rb_2015_2025_df.to_csv(full_file_path, index=False)